# Импорты и получение данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
!wget https://raw.githubusercontent.com/Laggg/data--for--students/master/flight_delays.csv

--2021-05-22 14:09:07--  https://raw.githubusercontent.com/Laggg/data--for--students/master/flight_delays.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3488296 (3.3M) [text/plain]
Saving to: ‘flight_delays.csv.1’

flight_delays.csv.1 100%[===================>]   3.33M  --.-KB/s    in 0.1s    

2021-05-22 14:09:08 (24.2 MB/s) - ‘flight_delays.csv.1’ saved [3488296/3488296]



In [ ]:
finemane = 'flight_delays.csv'
df = pd.read_csv(finemane, sep=',')

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


# Задание

### 1. Доля всех задержек ко всем вылетам

In [ ]:
group = pd.DataFrame(df.groupby('dep_delayed_15min').count()["Month"])
fig = px.pie(group, values='Month', names=df['dep_delayed_15min'].unique(), title='Delay proportion')
fig.show()

### 2. Зависимость количества задержек от длины пути, который предстоит пролететь самолёту

In [ ]:
df['dist_bin'] = df['Distance'] // 50
new = np.zeros((100, 2), dtype=np.int)
for i, j in df.groupby(['dist_bin', 'dep_delayed_15min']):
  new[i[0]][int(i[1] == 'N')] = j.shape[0]
#print(new)
hist = pd.DataFrame({"dist": range(0, 5000, 50), "prop_delays": new[:,0] / (new[:,0] + new[:,1])})
hist = hist[hist['prop_delays'].notna()]
print(hist)

fig = px.bar(hist, x='dist', y='prop_delays')
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in true_divide



    dist  prop_delays
0      0     0.143617
1     50     0.162381
2    100     0.137990
3    150     0.187500
4    200     0.176598
..   ...          ...
83  4150     0.125000
84  4200     0.148148
88  4400     0.500000
90  4500     0.083333
99  4950     0.181818

[75 rows x 2 columns]


### 3. Tоп 5 направлений, для которых чаще всего происходят задержки

In [ ]:
dest_dict = {}
for i, j in df.groupby(['Dest', 'dep_delayed_15min']):
  if i[0] not in dest_dict:
    dest_dict[i[0]] = [0, 0]
  dest_dict[i[0]][int(i[1] == 'N')] = j.shape[0]
for key, val in dest_dict.items():
  dest_dict[key] = val[0] / (val[0] + val[1])
hist = pd.DataFrame(list(dest_dict.items()), columns=['Dest', 'Ratio'])
hist = hist[hist['Ratio'].notna()]
hist = hist.sort_values(by='Ratio', ascending=False)
print(hist)

fig = px.bar(hist.head(5), x='Dest', y='Ratio')
fig.show()



    Dest     Ratio
271  TTN  1.000000
11   AKN  0.666667
31   BLI  0.666667
134  ILG  0.500000
188  MQT  0.500000
..   ...       ...
50   CDC  0.000000
266  TEX  0.000000
256  SOP  0.000000
258  SPS  0.000000
121  HKY  0.000000

[289 rows x 2 columns]


### 4. В какие времена года чаще всего происходят задержки рейсов

In [ ]:
month_dict = {}
for i, j in df.groupby(['Month', 'dep_delayed_15min']):
  if i[0] not in month_dict:
    month_dict[i[0]] = [0, 0]
  month_dict[i[0]][int(i[1] == 'N')] = j.shape[0]
for key, val in month_dict.items():
  month_dict[key] = val[0] / (val[0] + val[1])
hist = pd.DataFrame(list(month_dict.items()), columns=['Month', 'Ratio'])
hist = hist.sort_values(by='Ratio', ascending=False)
hist = hist[hist['Ratio'].notna()]

fig = px.bar(hist, x='Month', y='Ratio')
fig.show()


### 5. Топ 10 самых хороших перевозчиков, которые реже всего задерживают свои рейсы


In [ ]:
carrier_dict = {}
for i, j in df.groupby(['UniqueCarrier', 'dep_delayed_15min']):
  if i[0] not in carrier_dict:
    carrier_dict[i[0]] = [0, 0]
  carrier_dict[i[0]][int(i[1] == 'N')] = j.shape[0]
for key, val in carrier_dict.items():
  carrier_dict[key] = val[0] / (val[0] + val[1])
hist = pd.DataFrame(list(carrier_dict.items()), columns=['UniqueCarrier', 'Ratio'])
hist = hist[hist['Ratio'].notna()]
hist = hist.sort_values(by='Ratio', ascending=True)
print(hist)

fig = px.bar(hist.head(10), x='UniqueCarrier', y='Ratio')
fig.show()



   UniqueCarrier     Ratio
10            HA  0.043307
1             AQ  0.076923
11            HP  0.142235
16            TZ  0.143498
8             F9  0.155070
5             DH  0.161491
18            US  0.167387
13            NW  0.167890
6             DL  0.168922
15            OO  0.172801
20            XE  0.173869
14            OH  0.175882
4             CO  0.181818
0             AA  0.187938
17            UA  0.192263
12            MQ  0.194545
19            WN  0.213433
3             B6  0.213819
21            YV  0.224624
9             FL  0.242185
2             AS  0.243474
7             EV  0.275318


### 6. Топ 10 самых безответственных аэропортов, в которых чаще всего происходят задержки


In [ ]:
origin_dict = {}
for i, j in df.groupby(['Origin', 'dep_delayed_15min']):
  if i[0] not in origin_dict:
    origin_dict[i[0]] = [0, 0]
  origin_dict[i[0]][int(i[1] == 'N')] = j.shape[0]
for key, val in origin_dict.items():
  origin_dict[key] = val[0] / (val[0] + val[1])
hist = pd.DataFrame(list(origin_dict.items()), columns=['Origin', 'Ratio'])
hist = hist[hist['Ratio'].notna()]
hist = hist.sort_values(by='Ratio', ascending=False)
print(hist)

fig = px.bar(hist.head(10), x='Origin', y='Ratio')
fig.show()



    Origin     Ratio
117    GST  1.000000
8      ADK  1.000000
12     AKN  1.000000
165    LWB  0.750000
41     BRW  0.714286
..     ...       ...
215    PIE  0.000000
155    LCH  0.000000
135    ILG  0.000000
259    SPS  0.000000
216    PIH  0.000000

[289 rows x 2 columns]
